In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE


      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 colu

In [58]:
# --- 1. CARGA Y LIMPIEZA ---
df = pd.read_csv("Housing.csv")

# Convertir binarios
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
df[varlist] = df[varlist].apply(lambda x: x.map({'yes': 1, "no": 0}))

# Dummies para furnishingstatus
status = pd.get_dummies(df['furnishingstatus'], drop_first=True)
df = pd.concat([df, status], axis=1)
df.drop(['furnishingstatus'], axis=1, inplace=True)

In [ ]:

# 2_1. PREPROCESAMIENTO (Limpieza)
# Convertir binarios (yes/no) a (1/0)
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
df[varlist] = df[varlist].apply(lambda x: x.map({'yes': 1, "no": 0}))

In [59]:
# --- 2. DIVISIÓN ---
df_train, df_test = train_test_split(df, train_size=0.7, test_size=0.3, random_state=100)

In [61]:
# --- 3. SELECCIÓN DE VARIABLES (RFE) ---
# Entrenamos un RFE rápido para saber qué 10 variables son las mejores
X_temp = df_train.drop('price', axis=1)
y_temp = df_train['price']
lm_temp = LinearRegression()
rfe = RFE(estimator=lm_temp, n_features_to_select=10)
rfe.fit(X_temp, y_temp)
cols_10 = X_temp.columns[rfe.support_].tolist()

In [62]:
# --- 4. ESCALADO FINAL (Solo con las 10 elegidas) ---
X_train_final = df_train[cols_10]
y_train_final = df_train[['price']]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# ¡IMPORTANTE! Ajustar los escaladores con los datos reales
X_train_scaled = pd.DataFrame(scaler_X.fit_transform(X_train_final), columns=cols_10)
y_train_scaled = scaler_y.fit_transform(y_train_final)

In [63]:
# --- 5. ENTRENAMIENTO DEL MODELO FINAL ---
modelo_final = LinearRegression()
modelo_final.fit(X_train_scaled, y_train_scaled)

LinearRegression()

In [64]:
# --- 6. GUARDAR TODO ---
joblib.dump(modelo_final, 'modelo_housing.pkl')
joblib.dump(scaler_X, 'escalador_X.pkl')
joblib.dump(scaler_y, 'escalador_y.pkl')
joblib.dump(cols_10, 'columnas_seleccionadas.pkl')

print("¡Éxito! Archivos guardados y sincronizados.")
print("Columnas seleccionadas:", cols_10)

¡Éxito! Archivos guardados y sincronizados.
Columnas seleccionadas: ['bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'unfurnished']


In [ ]:
Estas NO NO NO
# 2. PREPROCESAMIENTO (Limpieza)
# Convertir binarios (yes/no) a (1/0)
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
df[varlist] = df[varlist].apply(lambda x: x.map({'yes': 1, "no": 0}))

In [47]:
# Variables Dummy (furnishingstatus)
status = pd.get_dummies(df['furnishingstatus'], drop_first=True)
df = pd.concat([df, status], axis=1)
df.drop(['furnishingstatus'], axis=1, inplace=True)

In [48]:
# 3. DIVISIÓN DEL DATASET
df_train, df_test = train_test_split(df, train_size=0.7, test_size=0.3, random_state=100)

In [49]:
# 4. ESCALADO (La clave para evitar el KeyError)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Columnas numéricas a escalar (X)
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

# Escalamos las características (X)
df_train[num_vars] = scaler_X.fit_transform(df_train[num_vars])
df_test[num_vars] = scaler_X.transform(df_test[num_vars])

# Escalamos el precio (y) - Usamos pd.DataFrame para que el escalador no falle
y_train = scaler_y.fit_transform(df_train[['price']])
y_test = scaler_y.transform(df_test[['price']])

# Definimos X (quitando el precio del dataframe de entrenamiento)
X_train = df_train.drop('price', axis=1)
X_test = df_test.drop('price', axis=1)

In [50]:
# 5. SELECCIÓN DE CARACTERÍSTICAS (RFE)
lm = LinearRegression()
rfe = RFE(estimator=lm, n_features_to_select=10)
rfe.fit(X_train, y_train)

X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_test_rfe = X_test[X_test.columns[rfe.support_]]

In [51]:
# 6. ENTRENAMIENTO FINAL
lm_final = LinearRegression()
lm_final.fit(X_train_rfe, y_train)

LinearRegression()

In [52]:
# 7. PREDICCIÓN Y DESESCALADO (Convertir a dinero real)
# Supongamos una nueva casa (valores entre 0 y 1 ya escalados para el ejemplo)
# El orden debe coincidir con X_train_rfe.columns
nombres_columnas = X_train_rfe.columns
casa_nueva_datos = [[0.5, 1, 1, 0.4, 1, 0, 0, 1, 0, 1]] 
X_nueva_casa = pd.DataFrame(casa_nueva_datos, columns=nombres_columnas)

# Predecir (da un resultado entre 0 y 1)
pred_escalada = lm_final.predict(X_nueva_casa)

# DESESCALAR para ver el precio real
precio_final = scaler_y.inverse_transform(pred_escalada)

print(f"--- RESULTADO FINAL ---")
print(f"El valor estimado de la casa es: ${precio_final[0][0]:,.2f}")

--- RESULTADO FINAL ---
El valor estimado de la casa es: $8,856,292.07


In [53]:
import joblib

# Definimos los nombres de los archivos
files = {
    'modelo': 'modelo_housing.pkl',
    'scaler_X': 'escalador_X.pkl',
    'scaler_y': 'escalador_y.pkl',
    'columnas': 'columnas_seleccionadas.pkl'
}

# Guardamos cada objeto
joblib.dump(lm_final, files['modelo'])
joblib.dump(scaler_X, files['scaler_X'])
joblib.dump(scaler_y, files['scaler_y'])
joblib.dump(list(X_train_rfe.columns), files['columnas'])

print("¡Modelo y componentes guardados exitosamente!")

¡Modelo y componentes guardados exitosamente!


In [57]:
import joblib
from sklearn.preprocessing import MinMaxScaler

# 1. Obtenemos las 10 columnas que RFE seleccionó
columnas_finales = X_train.columns[rfe.support_].tolist()

# 2. Creamos y ENTRENAMOS el escalador específicamente para esas 10 columnas
# USAMOS X_train[columnas_finales] SIN ESCALAR para que aprenda los rangos reales
sc_X_final = MinMaxScaler()
sc_X_final.fit(X_train[columnas_finales]) 

# 3. Guardamos TODO de nuevo
joblib.dump(lm_final, 'modelo_housing.pkl')
joblib.dump(sc_X_final, 'escalador_X.pkl') # Este es el que ahora sí funcionará
joblib.dump(scaler_y, 'escalador_y.pkl')
joblib.dump(columnas_finales, 'columnas_seleccionadas.pkl')

print("¡Escalador corregido y guardado!")

¡Escalador corregido y guardado!


In [56]:
# --- Ejecuta esto en tu script de entrenamiento ---

# 1. Identificar las 10 columnas que eligió el RFE
cols_finales = X_train.columns[rfe.support_].tolist()

# 2. Crear un NUEVO escalador solo para esas 10 columnas
# Esto es vital para que el escalador reconozca 'airconditioning', 'mainroad', etc.
scaler_X_final = MinMaxScaler()
scaler_X_final.fit(X_train[cols_finales]) # Entrenar con los datos sin escalar de esas 10 cols

# 3. Guardar los objetos actualizados
import joblib
joblib.dump(lm_final, 'modelo_housing.pkl')
joblib.dump(scaler_X_final, 'escalador_X.pkl') # Ahora este conoce las 10 columnas
joblib.dump(scaler_y, 'escalador_y.pkl')
joblib.dump(cols_finales, 'columnas_seleccionadas.pkl')

print("Componentes sincronizados y guardados.")
print("Las columnas que el modelo espera son:", cols_finales)

Componentes sincronizados y guardados.
Las columnas que el modelo espera son: ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']
